# Bioprocess data extraction

Extract specific bioprocess data from metapath dataset.

In [1]:
import duckdb

metapath_ids = "../bioprocess_metapath_to_gene_pval_and_dwpc/data/sources/metapaths.csv"
metapath_data = "./data/connectivity-search-precalculated-metapath-data.parquet"

In [2]:
with duckdb.connect() as ddb:
    # gather the metapath ids as a SQL formatted list
    ids = (
        "'"
        + "','".join(
            ddb.execute(
                f"""
                SELECT *
                FROM read_csv('{metapath_ids}')
                """
            )
            .df()["metapath"]
            .to_list()
        )
        + "'"
    )

    # use the list to search the full dataset, returning the results
    # which match the metapath ids
    metapath_results = ddb.execute(
        f"""
        SELECT *
        FROM read_parquet('{metapath_data}')
        WHERE metapath_id in ({ids})
        """
    ).df()

metapath_results

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,id,source_identifier,target_identifier,metapath_id,path_count,adjusted_p_value,p_value,dwpc,source_degree,target_degree,n_dwpcs,n_nonzero_dwpcs,nonzero_mean,nonzero_sd,source_id,target_id,dgp_id
0,3131035,GO:0000012,10396,BPpGbCuG,1,8.802685e-03,1.872912e-04,7.448745,7,1,87936800,1004171,4.843760,1.088758,21400,14539,12232813
1,3131036,GO:0000012,80223,BPpGbCuG,1,8.802685e-03,1.872912e-04,7.448745,7,1,87936800,1004171,4.843760,1.088758,21400,32981,12232813
2,3131037,GO:0000023,5058,BPpGbCuG,2,7.344636e-03,1.562689e-04,8.200729,3,60,860800,120487,4.329199,1.037640,10227,32281,12232604
3,3131038,GO:0000023,55837,BPpGbCuG,2,6.697539e-04,1.425008e-05,8.965426,3,13,8177600,401106,4.369826,1.059864,10227,19609,12232561
4,3131039,GO:0000027,662,BPpGbCuG,1,1.492915e-02,3.176414e-04,7.269739,22,1,22041600,755044,4.318069,1.082651,22556,45843,12233803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48150835,161055434,GO:1903047,54984,BPpGeAeG,18268,3.231764e-10,6.876094e-12,2.994781,722,41,48000,48000,2.826494,0.024426,24964,44809,14615383
48150836,161055476,GO:1903047,55036,BPpGeAeG,9242,4.163348e-16,8.858188e-18,2.645025,722,18,61800,61800,2.435013,0.024018,24964,10346,14615360
48150837,161055586,GO:1903047,55184,BPpGeAeG,8244,1.996607e-16,4.248099e-18,2.647178,722,18,61800,61800,2.435013,0.024018,24964,36555,14615360
48150838,161055623,GO:1903047,55225,BPpGeAeG,9869,3.369435e-18,7.169010e-20,2.658797,722,18,61800,61800,2.435013,0.024018,24964,41427,14615360


In [3]:
# export the results to parquet
metapath_results.to_parquet("data/bp_subset_metapaths.parquet")